In [1]:
import os
os.chdir("..")

import os

os.environ["CONFIG_APP_DIR"] = "config"
import json
import re

import random

random.randint(1, 100000)

from etl_pipeline.service.proto.api.etl_pipeline_pb2_grpc import EtlPipelineServiceStub
from etl_pipeline.config import ConsulServiceConfig
from etl_pipeline.service.proto.api.etl_pipeline_pb2 import (
    FAILURE,
    SUCCESS,
    Alert,
    Match,
    RunEtlRequest,
)


def load_alert(filepath: str = "tests/shared/sample/wm_address_in_payload_format.json"):
    with open(filepath, "r") as f:
        text = json.load(f)
        ids = [
            text[i]
            for key in text
            for i in re.compile(r".*matchRecords\[\d+\].matchId.*").findall(key)
            if i
        ]
        matches_ids = [
            Match(match_id=i, match_name=f"alerts/2/matches/{random.randint(1, 100000)}") for num, i in enumerate(ids)
        ]
        alert = Alert(batch_id="1", alert_name="alerts/2", matches=matches_ids)
        for key, value in text.items():
            alert.flat_payload[str(key)] = str(value)
    return alert

request_alert = load_alert("tests/shared/sample/big_fat_flat_payload.json")

In [33]:
import grpc
service_config = ConsulServiceConfig()
with open(service_config.grpc_client_tls_ca, "rb") as f:
    ca = f.read()
with open(service_config.grpc_client_tls_private_key, "rb") as f:
    private_key = f.read()
with open(service_config.grpc_client_tls_public_key_chain, "rb") as f:
    certificate_chain = f.read()
server_credentials = grpc.ssl_channel_credentials(ca, private_key, certificate_chain)
channel = grpc.insecure_channel("10.8.0.3:27762")

In [34]:
import grpc
# channel = grpc.insecure_channel("localhost:9090",  options=[
#             ("grpc.max_send_message_length", 27838500),
#             ("grpc.max_receive_message_length", 27838500),
#         ])
stub = EtlPipelineServiceStub(channel)
from time import time
start= time()
%timeit 
time() - start

9.751319885253906e-05

In [35]:
response = stub.RunEtl(RunEtlRequest(alerts=[load_alert("tests/shared/sample/big_fat_flat_payload.json") for i in range(1)]), timeout=60)

In [36]:
response

etl_alerts {
  batch_id: "1"
  alert_name: "alerts/2"
  etl_status: SUCCESS
  etl_matches {
    match_name: "alerts/2/matches/83868"
  }
  etl_matches {
    match_name: "alerts/2/matches/19993"
  }
  etl_matches {
    match_name: "alerts/2/matches/90747"
  }
  etl_matches {
    match_name: "alerts/2/matches/71167"
  }
  etl_matches {
    match_name: "alerts/2/matches/64348"
  }
  etl_matches {
    match_name: "alerts/2/matches/86672"
  }
  etl_matches {
    match_name: "alerts/2/matches/78545"
  }
  etl_matches {
    match_name: "alerts/2/matches/26329"
  }
  etl_matches {
    match_name: "alerts/2/matches/72581"
  }
}

In [28]:
from itertools import zip_longest
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [29]:
list(zip([1,2], [3,4], grouper([1,2], 2)))

[(1, 3, (1, 2))]

In [1]:
def load_alert(filepath: str = "tests/shared/sample/wm_address_in_payload_format.json"):
    with open(filepath, "r") as f:
        text = json.load(f)
        import pdb; pdb.set_trace()
        matches = [Match(match_id="0", match_name=f"alerts/2/matches/{num}")for num, _ in enumerate([re.findall(r"matchRecords\[\d+\]", key)[0] for key in text if re.search(r"matchRecords\[\d+\]", key)])]
        alert = Alert(batch_id="1", alert_name="alerts/2", matches=matches)
        for key, value in text.items():
            alert.flat_payload[str(key)] = str(value)
    return alert


In [2]:
with open("tests/shared/sample/wm_address_in_payload_format.json", "r") as f:
        text = json.load(f)
        

FileNotFoundError: [Errno 2] No such file or directory: 'tests/shared/sample/wm_address_in_payload_format.json'

In [14]:
import re

['49159509', '49159506']

In [16]:
matches

[match_id: "0"
 match_name: "alerts/2/matches/0",
 match_id: "0"
 match_name: "alerts/2/matches/1"]